## 통계 기반 기법

말뭉치 전처리 과정: Text를 단어로 자른 뒤 고유의 ID를 붙여주는 과정이다.

In [1]:
text = 'You say goodbye and I say hello'
print('text 확인', text)
#소문자로 변경
text = text.lower()
#마침표 처리
text = text.replace('.', ' .')
#Text => Word
words = text.split(' ')
print('단어 확인',words)

#말뭉치 전처리 과정 Vocab 사전 만드는 과정
word_to_id = {}
id_to_word = {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word
        
print('id_to_word 확인',id_to_word)
print('word_to_id 확인',word_to_id)

# Text 말뭉치 전처리 결과
import numpy as np
corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
print('결과 확인', corpus)

text 확인 You say goodbye and I say hello
단어 확인 ['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello']
id_to_word 확인 {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello'}
word_to_id 확인 {'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5}
결과 확인 [0 1 2 3 4 1 5]


위의 과정을 한번에 해결하는 Method이다.  
return 으로서 word_to_id, id_to_word, corpus를 Return하므로 위의 과정을 한번에 처리할 수 있다.

In [2]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

위에서 선언한 Method를 확인

In [3]:
text = 'You say goodbye and I say hello'
corpus, word_to_id, id_to_word = preprocess(text)
print('corpus: ',corpus)
print('word_to_id: ',word_to_id)
print('id_to_word: ',id_to_word)

corpus:  [0 1 2 3 4 1 5]
word_to_id:  {'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5}
id_to_word:  {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello'}


동시 발생 행렬 선언  
<table class="table">
	<tbody>
	<tr>
		<td></td>
		<td>you</td>
		<td>say</td>
		<td>goodbye</td>
		<td>and</td>
		<td>i</td>
		<td>hello</td>
		<td>.</td>
	</tr>
	<tr>
		<td>you</td>
		<td>0</td>
		<td>1</td>
		<td>0</td>
		<td>0</td>
		<td>0</td>
		<td>0</td>
		<td>0</td>
	</tr>
	<tr>
		<td>say</td>
		<td>1</td>
		<td>0</td>
		<td>1</td>
		<td>0</td>
		<td>1</td>
		<td>1</td>
		<td>0</td>
	</tr>
	<tr>
		<td>goodbye</td>
		<td>0</td>
		<td>1</td>
		<td>0</td>
		<td>1</td>
		<td>0</td>
		<td>0</td>
		<td>0</td>
	</tr>
	<tr>
		<td>and</td>
		<td>0</td>
		<td>0</td>
		<td>1</td>
		<td>0</td>
		<td>1</td>
		<td>0</td>
		<td>0</td>
	</tr>
	<tr>
		<td>i</td>
		<td>0</td>
		<td>1</td>
		<td>0</td>
		<td>1</td>
		<td>0</td>
		<td>0</td>
		<td>0</td>
	</tr>
	<tr>
		<td>hello</td>
		<td>0</td>
		<td>1</td>
		<td>0</td>
		<td>0</td>
		<td>0</td>
		<td>0</td>
		<td>1</td>
	</tr>
	<tr>
		<td>.</td>
		<td>0</td>
		<td>0</td>
		<td>0</td>
		<td>0</td>
		<td>0</td>
		<td>1</td>
		<td>0</td>
	</tr>

</tbody>
</table>
<br>

In [4]:
C = np.array([
    [0,1,0,0,0,0,0],
    [1,0,1,0,1,1,0],
    [0,1,0,1,0,0,0],
    [0,0,1,0,1,0,0],
    [0,1,0,1,0,0,0],
    [0,1,0,0,0,0,1],
    [0,0,0,0,0,1,0],
],dtype=np.int32)
print(C[0])
print(C[1])
print(C[2])

[0 1 0 0 0 0 0]
[1 0 1 0 1 1 0]
[0 1 0 1 0 0 0]


아래 Method는 위와 같은 과정을 수행하는 Method이다.
- corpus: 말뭉치
- vocab_size: vocab_size
- window_size: window_size

In [5]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix

**코사인 유사도 식**  
<p>$$similarity(x,y) = \frac{x y}{||x|| ||y||} = \frac{x_1y_1 + ... + x_ny_n}{\sqrt{x_1^2 + ... + x_n^2}\sqrt{y_1^2 + ... + y_n^2}}$$</p>
위의 식은 -1 ~ 1의 값을 가지게 되고 각 값에 대한 그림은 아래와 같다.  
<div><img src="https://wikidocs.net/images/page/24603/%EC%BD%94%EC%82%AC%EC%9D%B8%EC%9C%A0%EC%82%AC%EB%8F%84.PNG" height="250" width="600" /></div><br>
즉 각각의 성분이 얼만큼 유사한지를 계산하는 과정이다.  

In [6]:
def cos_similarity(x, y, eps=1e-8):
    nx = x / (np.sqrt(np.sum(x ** 2)) + eps)
    ny = y / (np.sqrt(np.sum(y ** 2)) + eps)
    return np.dot(nx, ny)

위의 Method를 활용하여 Text= "You say goodbye and I say hello"에서 You와 I의 유사도를 비교하는 과정이다.  

In [7]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

c0 = C[word_to_id['you']]  # "you"의 단어 벡터
c1 = C[word_to_id['i']]    # "i"의 단어 벡터
print(cos_similarity(c0, c1))

0.7071067691154799


위에서 선언한 Method를 개선하여 검색어와 가장 비슷한 단어를 추출하는 Method를 만들어보자. 
- query: 검색어
- word_to_id: 단어에서 단어 ID로의 딕셔너리
- id_to_word: 단어 ID에서 단어로의 딕셔너리
- word_matrix: 단어 벡터들을 한데 모은 행렬, 각 행에는 대응하는 단어의 벡터가 저장되어있다고 가정
- top: 상위 몇 개까지 추출할지에 관한 Parameter
- argsort(): 배열에 담긴 원소의 인덱스를 내림차순으로 정리

In [8]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    if query not in word_to_id:
        print('%s(을)를 찾을 수 없습니다.' % query)
        return

    print('\n[query] ' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    # 코사인 유사도 계산
    vocab_size = len(id_to_word)

    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    # 코사인 유사도를 기준으로 내림차순으로 출력
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))

        count += 1
        if count >= top:
            return
most_similar('you',word_to_id,id_to_word,C,top=5)


[query] you
 goodbye: 0.7071067691154799
 i: 0.7071067691154799
 hello: 0.7071067691154799
 say: 0.0
 and: 0.0


위와 같이 단어 사이의 거리만으로서 판단하는 것은 매우 큰 문제점이 발생하게 된다.  
영어에서의 **the**와같이 많이 나오는 수식어는 자주 출몰하게되어서 모든 단어와의 거리가 가까워 지게 되고 이로 인하여 많은 단어와의 상관성이 높에 나올 수 있다.  
이러한 문제점을 해결한 방안이 **PMI**이다.  
**PMI 식**  
<p>$$log_2 \frac{P(x,y)}{P(x)P(y)} = log_2 \frac{\frac{C(x,y)}{N}}{\frac{C(x)}{N}\frac{C(y)}{N}} = log_2 \frac{C(x,y)N}{C(x)C(y)}$$</p>

위의 식에서의 문제는 P(x,y) = 0 인경우 <span>$$log_20 =-\inf$$ </span>이므로 문제가 발생하게 된다. 위와 같은 문제를 해결한 것이 **PPMI**이고 아래 식과 같다.  
**PPMI 식**
<p>$$PPMI(x,y) = max(0,PMI(x,y))$$</p>

In [9]:
def ppmi(C, eps = 1e-8):
    #동시 발생 행렬과 같은 크기의 행렬
    M = np.zeros_like(C, dtype=np.float32)
    #말뭉치에 포함된 단어 수
    N = np.sum(C)
    #각 단어의 발생 횟수
    S = np.sum(C, axis=0)
    
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            #PMI 식
            pmi = np.log2(C[i, j] * N / (S[j]*S[i]) + eps)
            #PPMI 식
            M[i, j] = max(0, pmi)

    return M
W = ppmi(C)

#유효 자릿수를 3자리로 표시
np.set_printoptions(precision=3)
print('동시 발생 행렬')
print(C)
print('-'*50)
print('PPMI')
print(W)

동시 발생 행렬
[[0 1 0 0 0 0 0]
 [1 0 1 0 1 1 0]
 [0 1 0 1 0 0 0]
 [0 0 1 0 1 0 0]
 [0 1 0 1 0 0 0]
 [0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0]]
--------------------------------------------------
PPMI
[[0.    1.807 0.    0.    0.    0.    0.   ]
 [1.807 0.    0.807 0.    0.807 0.807 0.   ]
 [0.    0.807 0.    1.807 0.    0.    0.   ]
 [0.    0.    1.807 0.    1.807 0.    0.   ]
 [0.    0.807 0.    1.807 0.    0.    0.   ]
 [0.    0.807 0.    0.    0.    0.    2.807]
 [0.    0.    0.    0.    0.    2.807 0.   ]]


#### 차원 감소
위와 같은 문제를 위하여 차원감소 방법중에 SVD를 사용한다.  
SVD에 자세한 내용은 아래 링크 참조  
<a href="https://wjddyd66.github.io/ai/2019/07/24/PCA.html">SVD 자세한 내용</a><br>
위와 같은 식은 **numpy의 ilnalg Module**을 사용하여 구현한다.  

In [10]:
import matplotlib.pyplot as plt

# SVD
U, S, V = np.linalg.svd(W)

print('동시 발생 행렬')
print(C[0])
print('-'*10)
print('PPMI')
print(W[0])
print('-'*10)
print('SVD')
print(U[0])

# 플롯
for word, word_id in word_to_id.items():
    plt.annotate(word, (U[word_id, 0], U[word_id, 1]))
plt.scatter(U[:,0], U[:,1], alpha=0.5)
plt.show()

동시 발생 행렬
[0 1 0 0 0 0 0]
----------
PPMI
[0.    1.807 0.    0.    0.    0.    0.   ]
----------
SVD
[ 0.000e+00  3.409e-01 -3.886e-16 -1.205e-01  9.323e-01 -1.110e-16
 -1.467e-16]


<Figure size 640x480 with 1 Axes>

PTB 데이터 셋을 활용, sklearn의 SVD를 활용하여 선언한 Method의 결과를 확인하여 보자.

In [11]:
from dataset import ptb


window_size = 2
wordvec_size = 100

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
print('동시발생 수 계산 ...')
C = create_co_matrix(corpus, vocab_size, window_size)
print('PPMI 계산 ...')
W = ppmi(C)

print('calculating SVD ...')
try:
    # truncated SVD (빠르다!)
    from sklearn.utils.extmath import randomized_svd
    U, S, V = randomized_svd(W, n_components=wordvec_size, n_iter=5,
                             random_state=None)
except ImportError:
    # SVD (느리다)
    U, S, V = np.linalg.svd(W)

word_vecs = U[:, :wordvec_size]

querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)

동시발생 수 계산 ...
PPMI 계산 ...


KeyboardInterrupt: 